In [1]:
import praw
import datetime
import csv

In [2]:
user_agent = "Test/0.1 by maxoboe"
with open('account_info.txt', 'r') as file:
    lines = file.read().splitlines()
    username = lines[0]
    password = lines[1]
    clientID = lines[2]
    secretID = lines[3]
reddit = praw.Reddit(client_id=clientID,
                     client_secret=secretID,
                     user_agent=user_agent)

with open('climate_submissions.csv', 'r', encoding='utf-8') as infile, \
    open('climate_comments.csv', 'w+', encoding='utf-8', newline='') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['title', 'date', 'time', 'comment']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in reader:
        if row['comments'] is '0':
            continue
        submission = reddit.submission(id=row['id'])
        submission.comments.replace_more(limit=None)
        for top_level_comment in submission.comments:
            dateTime = datetime.datetime.fromtimestamp(top_level_comment.created)
            writer.writerow({'title':submission.title, 'date': dateTime.date(), 'time': dateTime.time(), 
                             'comment': top_level_comment.body})